In [1]:
import numpy as np
import pandas as pd
import math
from simpledbf import Dbf5
import xarray as xr
import rasterio

In [8]:
#field data is 0-20cm reference depth 

# import datasets from all horizons
d01 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\Field_data\\krig_field_data_attr_corr1.dbf"
# convert to pandas dataframes
dbf = Dbf5(d01)

df = dbf.to_dataframe()

In [9]:
df.head()

,Join_Count,TARGET_FID,Id,sand,clay,silt,org_c,ph,slope,cf
0,3,0,0,831148.0,64296.0,52469.0,2909.0,52624.0,37187.0,4.0
1,1,1,0,874946.0,75612.0,52229.0,3166.0,55136.0,33785.0,1.0
2,3,2,0,905178.0,80479.0,58097.0,3664.0,56144.0,44407.0,1.0
3,2,3,0,893643.0,80527.0,52455.0,3703.0,56284.0,626.0,2.0
4,3,4,0,798011.0,72017.0,60633.0,3392.0,55116.0,63817.0,16.0


In [10]:
# start by getting all values in the correct units 
# divide texture and ph by 1000000, divide slope by 10000, calculate OM, divide cf by 100
df['sand'] = df['sand']/10000
df['silt'] = df['silt']/10000
df['clay'] = df['clay']/10000
df['slope'] = df['slope']/10000
df['om'] = (df['org_c']/10000)*2.17
df['gravel'] = df['cf']/100
df['ph'] = df['ph']/10000

#fix textures so that they add to 100 since sand, silt and clay were krigged independently
df['total_p'] = df['sand']+ df['silt'] + df['clay']
df['sand'] = df['sand']/df['total_p']
df['silt'] = df['silt']/df['total_p']
df['clay'] = df['clay']/df['total_p']

In [11]:
df.head()

,Join_Count,TARGET_FID,Id,sand,clay,silt,org_c,ph,slope,cf,om,gravel,total_p
0,3,0,0,0.876819,0.067829,0.055352,2909.0,5.2624,3.7187,4.0,0.631253,0.04,94.7913
1,1,1,0,0.872514,0.075402,0.052084,3166.0,5.5136,3.3785,1.0,0.687022,0.01,100.2787
2,3,2,0,0.867233,0.077105,0.055662,3664.0,5.6144,4.4407,1.0,0.795088,0.01,104.3754
3,2,3,0,0.870467,0.078439,0.051095,3703.0,5.6284,0.0626,2.0,0.803551,0.02,102.6625
4,3,4,0,0.857467,0.077383,0.065150,3392.0,5.5116,6.3817,16.0,0.736064,0.16,93.0661


In [12]:
def texture(row):
    sand = row['sand']
    silt = row['silt']
    clay = row['clay']
    texture = ""

    if (sand >= .85 and silt <= .15 and clay <= .10):  # sand
            texture = "sand"
    elif (sand >= .70 and sand <= 90 and silt <= .30 and clay <= .15):  # loamy sand
            texture = "loamy sand"
    elif (sand >= .43 and sand <= .85 and silt <= .50 and clay <= .20):  # sandy loam
            texture = "sandy loam"
    elif (sand >= .23 and sand <= .52 and silt >= .28 and silt <= .50 and clay >= .7 and clay <= .27):  # loam
            texture = "loam"
    elif (sand <= .50 and silt >= .50 and silt <= .88 and clay <= .27):  # silt loam
            texture = "silt loam"
    elif (sand <= .20 and silt >= .88 and clay <= .12):  # silt
            texture = "silt"
    elif (sand >= .45 and sand <= .80 and silt <= .28 and clay >= .20 and clay <= .35):  # sandy clay loam
            texture = "sandy clay loam"
    elif (sand >= .20 and sand <= .45 and silt >= .15 and silt <= .53 and clay >= .27 and clay <= .40):  # clay loam
            texture = "clay loam"
    elif (sand <= .20 and silt >= .40 and silt <= .73 and clay >= .27 and clay <= .40):  # silty clay loam
            texture = "silty clay loam"
    elif (sand >= .45 and sand <= .65 and silt <= .20 and clay >= .35 and clay <= .45):  # sandy clay
            texture = "sandy clay"
    elif (sand <= .20 and silt >= .40 and silt <= .60 and clay >= .40 and clay <= .60):  # silty clay
            texture = "silty clay"
    elif (sand <= .45 and silt <= .40 and clay >= .40):  # clay
            texture = "clay"
    else: 
        texture = "problem"
    
    return texture

In [13]:
def awc(row):
    sand = row["sand"]
    silt = row["silt"]
    clay = row["clay"]
    om = row["om"]
    gravel = row["gravel"]

    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    AWC_final = LAWC*100
       
    return AWC_final

In [14]:
def LAWC_LCC(row):
    AWC = row['AWC']
    if AWC > 18:
        awc_lcc = 1
    elif AWC > 12 and AWC <= 18:
        awc_lcc = 2
    elif AWC > 6 and AWC <= 12:
        awc_lcc = 3
    elif AWC <= 6: 
        awc_lcc = 4
    else:
        awc_lcc = 1
    
    return awc_lcc

In [15]:
# function to calculate erosion risk LCC class
# calculates K factor and combines with slope to determine risk
def erosion_risk_LCC(row):
    texture = row["texture"]
    slope = row['slope']
    #first calculate k factor based on texture
    list1 = ["loam", "silt loam", "silt"]
    if list1.count(texture) > 0:
        k_fac = .33
    else: 
        k_fac = .31
    
    #then calculate lcc class of erosion risk combining k factor and slope
    if k_fac >.32 and slope <=2:
        erosion_risk_lcc = 1
    elif k_fac >.32 and (slope > 2 and slope <=5):
        erosion_risk_lcc = 2
    elif k_fac >.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 3
    elif k_fac >.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 4
    elif k_fac >.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 6    
    elif k_fac >.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac >.32 and slope > 60:
        erosion_risk_lcc = 8
    elif k_fac <= 0.32 and slope <= 5:
        erosion_risk_lcc = 1       
    elif k_fac <= 0.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 2 
    elif k_fac <= 0.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 3
    elif k_fac <= 0.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 4   
    elif k_fac <= 0.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac <= 0.32 and slope > 60:
        erosion_risk_lcc = 8
    else: 
        erosion_risk_lcc = 1
    
    return erosion_risk_lcc

In [16]:
# function to calculate surface soil texture LCC class
# takes in texture and outputs LCC class
def soil_tex_LCC(row):
    texture = row["texture"]
    list1 = ["sandy loam", "silt loam", "loam", "silt", "sandy clay loam", "silty clay loam", "clay loam"]
    list2 = ["sand", "loamy sand", "sandy clay", "silty clay"] 
    
    if list1.count(texture) > 0 :
        surface_st_lcc = 1
    elif list2.count(texture) > 0 :
        surface_st_lcc = 2
    elif texture == "clay":
        surface_st_lcc = 3
    else:
        surface_st_lcc = 1
    
    return surface_st_lcc

In [17]:
# function to calculate surface stoniness LCC class 
# takes in coarse fragments and outputs LCC class
def stoniness_LCC(row):
    stone = row['cf']
    
    if stone < .1:
        stone_LCC = 1
    elif stone >= .1 and stone < 3:
        stone_LCC = 2
    elif stone >= 3 and stone < 15:
        stone_LCC = 3
    elif stone >= 15 and stone < 50: 
        stone_LCC = 5
    elif stone >= 50 and stone < 90: 
        stone_LCC = 7
    elif stone >= 90:
        stone_LCC = 8
        
    return stone_LCC

In [18]:
# function to calculate permeability LCC class
# take in texture, calculate intermediary values, outpus LCC class
# based on landPKs alculation of permeability
def perm_LCC(row):
    sand = row['sand']
    silt = row['silt']
    clay = row['clay']
    om = row['om']
    gravel = row['gravel']
    
    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    SFC1 = 0.278*(sand) + 0.034*(clay) + 0.022*(om) - 0.018*(sand)*(om) - 0.027*(clay)*(om) - 0.584*(sand)*(clay) + 0.078
    SFC = SFC1 + ((0.636*SFC1) - 0.107)
    SASC = FC + SFC - 0.097*(sand) + 0.043
    MSD = (1- SASC)*2.65
    BSD = (1 - (gravel)) / (1 - (gravel)*(1 - 1.5*(MSD / 2.65)))
    L = (math.log(WP) - math.log(FC)) / (math.log(1500) - math.log(33))
    KS = 1930*(SASC)**(3-L)*BSD
    
    perm_lcc = 1
    if KS < 1.5:
        perm_lcc = 3
    elif KS >= 1.5 and KS <= 5:
        perm_lcc = 2
    elif KS > 5:
        perm_lcc = 1
        
    return perm_lcc

In [19]:
# function to calculate lime requirement LCC class
# takes in T_PH_H2O and outputs LCC class 
def lime_LCC(row):
    ph_val = row["ph"]
    if ph_val >= 5.5 and ph_val <= 7.2: 
        lime_lcc = 1
    elif (ph_val < 5.5 and ph_val >=4.5) or (ph_val > 7.2 and ph_val <= 8.4): 
        lime_lcc = 3
    elif ph_val < 4.5 or ph_val > 8.5:
        lime_lcc = 4
    
    return lime_lcc

In [20]:
df['texture'] = df.apply(texture, axis = 1)
df["AWC"] = df.apply(awc, axis=1)
df['erosion_risk_LCC'] = df.apply(erosion_risk_LCC, axis = 1)
df['soil_tex_LCC'] = df.apply(soil_tex_LCC, axis = 1)
df['stoniness_LCC'] = df.apply(stoniness_LCC, axis = 1)
df['perm_LCC'] = df.apply(perm_LCC, axis = 1)
df['lime_LCC'] = df.apply(lime_LCC, axis = 1)
df['AWC_LCC'] = df.apply(LAWC_LCC, axis = 1)

In [21]:
df.head()

,Join_Count,TARGET_FID,Id,sand,clay,silt,org_c,ph,slope,cf,...,gravel,total_p,texture,AWC,erosion_risk_LCC,soil_tex_LCC,stoniness_LCC,perm_LCC,lime_LCC,AWC_LCC
0,3,0,0,0.876819,0.067829,0.055352,2909.0,5.2624,3.7187,4.0,...,0.04,94.7913,sand,4.398946,1,2,3,1,3,4
1,1,1,0,0.872514,0.075402,0.052084,3166.0,5.5136,3.3785,1.0,...,0.01,100.2787,sand,4.586871,1,2,2,1,1,4
2,3,2,0,0.867233,0.077105,0.055662,3664.0,5.6144,4.4407,1.0,...,0.01,104.3754,sand,4.709825,1,2,2,1,1,4
3,2,3,0,0.870467,0.078439,0.051095,3703.0,5.6284,0.0626,2.0,...,0.02,102.6625,sand,4.597365,1,2,2,1,1,4
4,3,4,0,0.857467,0.077383,0.065150,3392.0,5.5116,6.3817,16.0,...,0.16,93.0661,sand,4.135262,2,2,5,1,1,4


In [23]:
#calculate final LCC and limitations
def calc_LCC(row):
    lcc_final = max(row["erosion_risk_LCC"], row["soil_tex_LCC"], row["stoniness_LCC"], row["lime_LCC"], row["perm_LCC"], row['AWC_LCC'])
    
    return lcc_final
def calc_LCC_no_awc(row):
    lcc_final = max(row["erosion_risk_LCC"], row["soil_tex_LCC"], row["stoniness_LCC"], row["lime_LCC"], row["perm_LCC"])
    
    return lcc_final

def calc_LCC_no_awc_nolim(row):
    lcc_final = max(row["erosion_risk_LCC"], row["soil_tex_LCC"], row["stoniness_LCC"], row["perm_LCC"])
    
    return lcc_final

def limitation(row):
    LCC_final = row['LCC_final']
    erosion = row['erosion_risk_LCC']
    tex = row['soil_tex_LCC']
    stone = row['stoniness_LCC']
    perm = row['perm_LCC']
    awc = row['AWC_LCC']
    lime = row['lime_LCC']

    
    lim = str(LCC_final)
    if LCC_final == erosion: 
        lim += "e "
    if LCC_final == tex: 
        lim +="s-t "
    if LCC_final == stone:
        lim += "s-r "
    if LCC_final == perm: 
        lim +="w-p "
    if awc == LCC_final: 
        lim +="s-a "
    if LCC_final == lime: 
        lim +="s-l "
    
    return(lim)

def limitation_noawc(row):
    LCC_final = row['LCC_final_noawc']
    erosion = row['erosion_risk_LCC']
    tex = row['soil_tex_LCC']
    stone = row['stoniness_LCC']
    perm = row['perm_LCC']
    lime = row['lime_LCC']

    
    lim = str(LCC_final)
    if LCC_final == erosion: 
        lim += "e "
    if LCC_final == tex: 
        lim +="s-t "
    if LCC_final == stone:
        lim += "s-r "
    if LCC_final == perm: 
        lim +="w-p "
    if LCC_final == lime: 
        lim +="s-l "
    
    return(lim)

def limitation_noawc_nolim(row):
    LCC_final = row['LCC_final_noawc_nolim']
    erosion = row['erosion_risk_LCC']
    tex = row['soil_tex_LCC']
    stone = row['stoniness_LCC']
    perm = row['perm_LCC']

    
    lim = str(LCC_final)
    if LCC_final == erosion: 
        lim += "e "
    if LCC_final == tex: 
        lim +="s-t "
    if LCC_final == stone:
        lim += "s-r "
    if LCC_final == perm: 
        lim +="w-p "
    
    return(lim)

In [24]:
df['LCC_final'] = df.apply(calc_LCC, axis = 1)
df['LCC_final_noawc'] = df.apply(calc_LCC_no_awc, axis = 1)
df['LCC_final_noawc_nolim'] = df.apply(calc_LCC_no_awc_nolim, axis = 1)

df['LCC_limitation'] = df.apply(limitation, axis = 1)
df['LCC_limitation_noawc'] = df.apply(limitation_noawc, axis = 1)
df['LCC_limitation_noawc_nolim'] = df.apply(limitation_noawc_nolim, axis = 1)

In [25]:
df

,Join_Count,TARGET_FID,Id,sand,clay,silt,org_c,ph,slope,cf,...,stoniness_LCC,perm_LCC,lime_LCC,AWC_LCC,LCC_final,LCC_final_noawc,LCC_final_noawc_nolim,LCC_limitation,LCC_limitation_noawc,LCC_limitation_noawc_nolim
0,3,0,0,0.876819,0.067829,0.055352,2909.0,5.2624,3.7187,4.0,...,3,1,3,4,4,3,3,4s-a,3s-r s-l,3s-r
1,1,1,0,0.872514,0.075402,0.052084,3166.0,5.5136,3.3785,1.0,...,2,1,1,4,4,2,2,4s-a,2s-t s-r,2s-t s-r
2,3,2,0,0.867233,0.077105,0.055662,3664.0,5.6144,4.4407,1.0,...,2,1,1,4,4,2,2,4s-a,2s-t s-r,2s-t s-r
3,2,3,0,0.870467,0.078439,0.051095,3703.0,5.6284,0.0626,2.0,...,2,1,1,4,4,2,2,4s-a,2s-t s-r,2s-t s-r
4,3,4,0,0.857467,0.077383,0.065150,3392.0,5.5116,6.3817,16.0,...,5,1,1,4,5,5,5,5s-r,5s-r,5s-r
5,3,5,0,0.858155,0.090605,0.051240,3350.0,5.2482,3.0948,1.0,...,2,1,3,4,4,3,2,4s-a,3s-l,2s-t s-r
6,4,6,0,0.855497,0.083138,0.061366,3196.0,5.3494,3.1746,6.0,...,3,1,3,4,4,3,3,4s-a,3s-r s-l,3s-r
7,4,7,0,0.881225,0.067156,0.051619,2919.0,5.3826,3.8286,2.0,...,2,1,3,4,4,3,2,4s-a,3s-l,2s-t s-r
8,2,8,0,0.851886,0.092717,0.055396,3089.0,5.3217,5.6513,0.0,...,1,1,3,4,4,3,2,4s-a,3s-l,2e s-t
9,3,9,0,0.873575,0.069714,0.056710,3326.0,5.5517,1.3385,3.0,...,3,1,1,4,4,3,3,4s-a,3s-r,3s-r


In [30]:
df['LCC_limitation_noawc_nolim'].value_counts()

3s-r           19301
2s-t s-r        8042
2s-t            1469
2e s-t s-r       812
5s-r             466
2e s-t           128
3e s-r            55
2s-r              46
4e                11
3e                 6
2e s-r             2
Name: LCC_limitation_noawc_nolim, dtype: int64

In [29]:
np.array(df['LCC_final_noawc_nolim'].value_counts())/30338

array([6.38209506e-01, 3.46067638e-01, 1.53602742e-02, 3.62581581e-04])

In [22]:
df.to_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\Field_Data\\field_data_LCC_updated.csv")

In [50]:
df['texture'].unique()

array(['loamy sand', 'sand', '', 'sandy loam', 'sandy clay loam'],
      dtype=object)

In [203]:
csv = pd.read_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\Field_Data\\field_data_LCC.csv")